### Import Required Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [4]:
%cd $PATH_TO_FOLDER

/content/drive/MyDrive/Name2Demographics


In [5]:
import sys  
sys.path.insert(0, 'Models/PreProcessing/')

from utils import *

sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

### Train Model on Data

In [6]:
cbse_df = CBSEData()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (84,88,90,94,96,100,102,106,108,112,114,118,120,123,125,128,130,134,136,140,142,145,147,151,153,157,159,163,165,169,171,175,177,180,182,185,187,191,193,194,196,200,202,206,208,211,213,216,218,222,224,228,230,234,236,240,242,246,248,252,254,257,259,263,265,269,271,275,277,281,283,287,289,293,295,299,301,305,307,311,313,317,319,323,325,328,330,334,336,340,342,346,348,351,353,354,356,359,361,364,366,370,372,376,378,381,383,387,389,393,395,399,401,405,407,411,413,417,419,423) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (82,86,88,92,94,98,100,104,106,110,112,116,124,128,130,134,136,140,142,146,148,152,154,158,160,164,166,170,172,176,178,182,184,188,190,194,196,200,202,206,208,212,214,218,220,224,226,230,232,233,235,239,241,245,247,2

In [7]:
train, val, test = splitTrainTestVal(cbse_df, 0.7 ,0, 'Name')

(train:(142259, 4), val:(0, 0), test:(62285, 4))


In [8]:
total_points = cbse_df.shape[0]
train_points = train.shape[0]
test_points = test.shape[0]
print(train_points/total_points)

0.695493390175219


In [9]:
sgd_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 6), lowercase = True, stop_words = "english")),
    ('tfidf', TfidfTransformer(sublinear_tf = True)),
    ('clf', SGDClassifier(class_weight="balanced",loss="squared_loss",max_iter=10000, random_state=42, validation_fraction = 0.3, average=1000
                          , learning_rate='adaptive', eta0=0.003))
])

In [10]:
sgd_pipe.fit(train['Name'].values.astype('str'), train['Gender'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:169: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  FutureWarning,


Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 6), stop_words='english')),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf',
                 SGDClassifier(average=1000, class_weight='balanced',
                               eta0=0.003, learning_rate='adaptive',
                               loss='squared_loss', max_iter=10000,
                               random_state=42, validation_fraction=0.3))])

In [11]:
y_pred=sgd_pipe.predict(test['Name'].values.astype('str'))
print(classification_report(test['Gender'], y_pred, target_names = ['Female', 'Male'], digits=4))

              precision    recall  f1-score   support

      Female     0.6166    0.6556    0.6355     24932
        Male     0.7600    0.7279    0.7436     37353

    accuracy                         0.6990     62285
   macro avg     0.6883    0.6918    0.6896     62285
weighted avg     0.7026    0.6990    0.7003     62285



### Save Model

In [12]:
%pwd

'/content/drive/My Drive/Name2Demographics'

In [13]:
filename = "Models/CBSEData/SGDClassifier/SavedModel/CBSEModel.pkl"
pickle.dump(sgd_pipe, open(filename, 'wb'))

### Load Model

In [ ]:
# filename = "SavedModels/FinalSet/SGDClassifier/CBSEModel.pkl"
# pipe = pickle.load(open(filename, 'rb'))

In [ ]:
# df, first_name = split_name_df(cbse_df, 'Name')

In [ ]:
# y_pred=pipe.predict(df['first_name'].values.astype('str'))
# print(classification_report(test['Gender'], y_pred, target_names = ['Male', 'Female']))

In [ ]:
# fn = pd.DataFrame()

In [ ]:
# fn['first_name'] = df['first_name']

In [ ]:
# df['predict'] = y_pred

In [ ]:
# fn['gender'] = y_pred
# fn['true_gender'] = df['Gender']

In [133]:
# print(fn.groupby(['first_name','gender']).size())

In [ ]:
# f1 = fn.drop_duplicates()
# f1 = f1.reset_index(drop=True)
# f1

,first_name,gender,true_gender
0,sarda,Girl,Girl
1,rakhi,Girl,Girl
2,surekha,Girl,Girl
3,shivani,Girl,Girl
4,ramdulari,Girl,Girl
...,...,...,...
25729,mannulal,Girl,Boy
25730,anada,Girl,Boy
25731,priyadarshan,Girl,Boy
25732,bindhyeshwari,Girl,Boy
